In [1]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from newsapi import NewsApiClient
from datetime import datetime, timedelta
import numpy as np
import requests
import nltk as nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
from nltk.corpus import stopwords, reuters
%matplotlib inline

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/samuelarciniega/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
load_dotenv()

True

In [3]:
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")
#print(alpaca_api_key)

In [4]:
api_key = os.getenv("news_api")
#print(api_key)

In [14]:
newsapi = NewsApiClient(api_key=api_key)

In [15]:
#15Min interval
timeframe = "1D"
start_date = pd.Timestamp(datetime.now(), tz="America/New_York").isoformat()
end_date = pd.Timestamp(datetime.now()- timedelta(30), tz="America/New_York").isoformat()
#today = pd.Timestamp("2021-01-02", tz="America/New_York").isoformat()
ticker = ["AAPL"]
apple_ticker = alpaca.get_barset(
    ticker,
    timeframe,
    limit=None,
    start=end_date,
    end=start_date,
    after=None,
    until=None,
).df
apple_ticker.dropna(inplace=True)
apple_ticker.head(20)

AAPL                                       
                             open     high       low     close    volume
time                                                                    
2021-06-03 00:00:00-04:00  124.66  124.850  123.1300  123.5550  68331470
2021-06-04 00:00:00-04:00  124.07  126.160  123.8500  125.8800  63613126
2021-06-07 00:00:00-04:00  126.18  126.320  124.8321  125.9100  62351961
2021-06-08 00:00:00-04:00  126.62  128.460  126.2101  126.7400  67231488
2021-06-09 00:00:00-04:00  127.21  127.750  126.5200  127.1600  46854275
2021-06-10 00:00:00-04:00  127.02  128.190  125.9400  126.1100  63377192
2021-06-11 00:00:00-04:00  126.53  127.440  126.1000  127.3521  46206013
2021-06-14 00:00:00-04:00  127.82  130.490  127.0700  130.4900  73169843
2021-06-15 00:00:00-04:00  129.88  130.600  129.3900  129.7400  53587032
2021-06-16 00:00:00-04:00  130.29  130.890  128.4610  130.1400  81577235
2021-06-17 00:00:00-04:00  129.80  132.550  129.6500  131.7800  82434812
2021-06-18 00:00:00-04:00  130.65  131.510  130.2400  130.4000  78114203
2021-06-21 00:00:00-04:00  130.32  132.410  129.2100  132.2900  63245869
2021-06-22 00:00:00-04:00  132.10  134.080  131.6200  133.9700  61664813
2021-06-23 00:00:00-04:00  133.78  134.320  133.2300  133.7000  54089974
2021-06-24 00:00:00-04:00  134.45  134.640  132.9300  133.4000  57263070
2021-06-25 00:00:00-04:00  133.47  133.890  132.8100  133.1100  42633100
2021-06-28 00:00:00-04:00  133.42  135.245  133.3500  134.7500  53815255
2021-06-29 00:00:00-04:00  134.82  136.490  134.3500  136.3300  55081959
2021-06-30 00:00:00-04:00  136.17  137.410  135.8700  136.9400  51014239

In [16]:
apple_df = pd.DataFrame()
apple_df["Apple_Close"] = apple_ticker["AAPL"]["close"]
#apple_df = apple_df.pct_change().dropna()
apple_df.index = apple_df.index.date 
apple_df.tail()

,Apple_Close
2021-06-28,134.75
2021-06-29,136.33
2021-06-30,136.94
2021-07-01,137.27
2021-07-02,139.96


In [17]:
apple_returns = apple_df.pct_change().dropna()
apple_returns.head(50)

,Apple_Close
2021-06-04,0.018818
2021-06-07,0.000238
2021-06-08,0.006592
2021-06-09,0.003314
2021-06-10,-0.008257
2021-06-11,0.009849
2021-06-14,0.024640
2021-06-15,-0.005748
2021-06-16,0.003083
2021-06-17,0.012602


In [18]:
def get_headlines(keyword):
    all_headlines = []
    all_dates = []    
    date = datetime.strptime(start_date[:10], "%Y-%m-%d")
    past_date = datetime.strptime(end_date[:10], "%Y-%m-%d")
    print(f"Fetching news about '{keyword}'")
    print("*" * 30)
    while date > past_date:
        print(f"retrieving news from: {date}")
        articles = newsapi.get_everything(
            q=keyword,
            from_param=str(date)[:10],
            to=str(date)[:10],
            language="en",
            sort_by="relevancy",
            page=1,
        )
        headlines = []
        for i in range(0, len(articles["articles"])):
            headlines.append(articles["articles"][i]["title"])
        all_headlines.append(headlines)
        all_dates.append(date)
        date = date - timedelta(days=1)
    return all_headlines, all_dates

In [19]:
inflation_headlines, dates = get_headlines("inflation")

Fetching news about 'inflation'
******************************
retrieving news from: 2021-07-03 00:00:00
retrieving news from: 2021-07-02 00:00:00
retrieving news from: 2021-07-01 00:00:00
retrieving news from: 2021-06-30 00:00:00
retrieving news from: 2021-06-29 00:00:00
retrieving news from: 2021-06-28 00:00:00
retrieving news from: 2021-06-27 00:00:00
retrieving news from: 2021-06-26 00:00:00
retrieving news from: 2021-06-25 00:00:00
retrieving news from: 2021-06-24 00:00:00
retrieving news from: 2021-06-23 00:00:00


NewsAPIException: {'status': 'error', 'code': 'rateLimited', 'message': 'You have made too many requests recently. Developer accounts are limited to 100 requests over a 24 hour period (50 requests available every 12 hours). Please upgrade to a paid plan if you need more requests.'}

In [20]:
RegTech_headlines,_ = get_headlines("security")

Fetching news about 'security'
******************************
retrieving news from: 2021-07-03 00:00:00


NewsAPIException: {'status': 'error', 'code': 'rateLimited', 'message': 'You have made too many requests recently. Developer accounts are limited to 100 requests over a 24 hour period (50 requests available every 12 hours). Please upgrade to a paid plan if you need more requests.'}

In [21]:
covid_headlines,_ = get_headlines("coronavirus")

Fetching news about 'coronavirus'
******************************
retrieving news from: 2021-07-03 00:00:00


NewsAPIException: {'status': 'error', 'code': 'rateLimited', 'message': 'You have made too many requests recently. Developer accounts are limited to 100 requests over a 24 hour period (50 requests available every 12 hours). Please upgrade to a paid plan if you need more requests.'}

In [23]:
sid = SentimentIntensityAnalyzer()

In [25]:
def headline_sentiment_summarizer_avg(headlines):
    sentiment = []
    for day in headlines:
        day_score = []
        for h in day:
            if h == None:
                continue
            else:
                day_score.append(sid.polarity_scores(h)["compound"])
        sentiment.append(sum(day_score) / len(day_score))
    return sentiment

In [27]:
inflation = headline_sentiment_summarizer_avg(inflation_headlines)
regtech = headline_sentiment_summarizer_avg(RegTech_headlines)
covid = headline_sentiment_summarizer_avg(covid_headlines)

NameError: name 'inflation_headlines' is not defined

In [16]:
topic_sentiments = pd.DataFrame(
    {
        "Inflation_avg": inflation,
        "RegTech_avg": regtech,
        "Covid_avg": covid,
    }
)

In [17]:
topic_sentiments.index = pd.to_datetime(dates)
topic_sentiments = apple_returns.join(topic_sentiments).dropna(how="any")
topic_sentiments.to_csv("topic_sentiment.csv")
display(topic_sentiments)

,Apple_Close,Inflation_avg,RegTech_avg,Covid_avg
2021-06-04,0.018818,-0.168915,0.093365,-0.090420
2021-06-07,0.000238,0.085070,-0.053740,0.067275
2021-06-08,0.006592,0.160855,-0.021220,0.003990
2021-06-09,0.003314,-0.052685,-0.268535,-0.026815
2021-06-10,-0.008257,-0.007730,-0.083360,0.063360
2021-06-11,0.009849,0.060070,0.206090,0.030350
2021-06-14,0.024640,0.160030,0.173350,-0.051515
2021-06-15,-0.005748,-0.026630,0.061830,-0.061655
2021-06-16,0.003083,0.068935,0.018640,-0.018875
2021-06-17,0.012602,0.061725,-0.072455,-0.016150


In [18]:
topic_sentiments.corr().style.background_gradient()

,Apple_Close,Inflation_avg,RegTech_avg,Covid_avg
Apple_Close,1.000000,0.089954,0.419371,0.034195
Inflation_avg,0.089954,1.000000,0.072733,0.388038
RegTech_avg,0.419371,0.072733,1.000000,0.081515
Covid_avg,0.034195,0.388038,0.081515,1.000000


In [9]:
topic_sentiments.plot()

NameError: name 'topic_sentiments' is not defined